Logs
- [2022/02/02]
  Scraping `pikiran-rakyat.com`.   
  The technique is the same as `report-antaranews.com`.  
  There is a difficulty in each url. It contains navigation page.  
  It means that, each news page is divided into several pages.

In [2]:
import requests
import xlsxwriter
import time
import datetime
import pickle
import re

import bs4
from bs4 import BeautifulSoup

import selenium
from seleniumwire import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
print(f"selenium.__version__: {selenium.__version__}")
bs4.__version__

selenium.__version__: 3.141.0


'4.9.3'

## Get a specific type of news

In [73]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--kiosk-printing')
#chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')

browser = webdriver.Chrome(
  executable_path="/home/henokh/Downloads/chromedriver_linux64/chromedriver", 
  options=chrome_options)

print("Go to the URL")
base_site = "https://www.pikiran-rakyat.com/search/?"
query = 'unjuk rasa'
query_full = f"q=" + "+".join(query.split())

page = 1
print(f"url: {base_site + query_full}")

browser.get(base_site + query_full)

Go to the URL
url: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa


In [74]:
print("Get all URLs (with its date and time) in each page")
elem = browser.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

soup_page = BeautifulSoup(source_code, parser="lxml")

[(url_.find("date", attrs={"class": "latest__date"}).text,
  url_.find("h4", attrs={"class": "latest__subtitle"}).text,
  url_.find("h2", attrs={"class": "latest__title"}).find('a')["href"]) 
    for url_ in soup_page.find_all("div", attrs={"class": "latest__right"})]

Get all URLs (with its date and time) in each page


[('31 Januari 2022, 18:02 WIB',
  'Jawa Barat',
  'https://www.pikiran-rakyat.com/jawa-barat/pr-013614173/ketua-umum-hingga-penunggang-patung-macan-lodaya-belasan-anggota-gmbi-ditetapkan-sebagai-tersangka'),
 ('29 Januari 2022, 07:41 WIB',
  'Internasional',
  'https://www.pikiran-rakyat.com/internasional/pr-013594340/unjuk-rasa-tuntut-hak-atas-air-berujung-ricuh-warga-iran-serang-truk-afghanistan'),
 ('29 Januari 2022, 06:49 WIB',
  'Jawa Barat',
  'https://www.pikiran-rakyat.com/jawa-barat/pr-013594019/dikumpulkan-di-polda-jateng-buntut-kerusuhan-di-polda-jabar-anggota-gmbi-alhamdulillah-dapat-makan'),
 ('28 Januari 2022, 15:52 WIB',
  'Jawa Barat',
  'https://www.pikiran-rakyat.com/jawa-barat/pr-013590001/polres-sumedang-sedihnya-jadi-anggota-gmbi-sumedang-anak-buah-susah-pimpinannya-ngumpet'),
 ('28 Januari 2022, 15:10 WIB',
  'Bandung Raya',
  'https://www.pikiran-rakyat.com/bandung-raya/pr-013589695/buntut-aksi-ricuh-di-bandung-polisi-tangkap-ketua-gmbi'),
 ('28 Januari 2022, 10:

## Iterate over all pages with condition on specific range of the date

In [16]:
page = 2

parent_div = browser.find_elements_by_xpath("//div[@class='paging__wrap clearfix']")
#print(f"   parent_ul: {parent_ul}")
element_list = parent_div[0].find_elements_by_xpath("//div[@class='paging__item']")
element_text = [element.text for element in element_list]

next_element = element_list[-1]
next_group_page_link = next_element.find_elements_by_xpath(".//a")[0].get_attribute("href")
print(f"   next_group_page_link: {next_group_page_link}")

   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=2


In [76]:
month_dict = {
  "Januari": "01", "Februari": "02", "Maret": "03",
  "April": "04", "Mei": "05", "Juni": "06",
  "Juli": "07", "Agustus": "08", "September": "09",
  "Oktober": "10", "November": "11", "Desember": "12"}
t_start = datetime.datetime(2019, 1, 1)

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--kiosk-printing')
#chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')

browser = webdriver.Chrome(
  executable_path="/home/henokh/Downloads/chromedriver_linux64/chromedriver", 
  options=chrome_options)

print("Go to the URL")
base_site = "https://www.pikiran-rakyat.com/search/?"
query = 'unjuk rasa'
query_full = f"q=" + "+".join(query.split())

page = 1
print(f"url: {base_site + query_full}")

browser.get(base_site + query_full)

print(f"Get all URLs (with its date and time) in page: {page:03d}")
elem = browser.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

soup_page = BeautifulSoup(source_code, parser="lxml")

all_urls = [
  (url_.find("date", attrs={"class": "latest__date"}).text,                      # date
   url_.find("h4", attrs={"class": "latest__subtitle"}).text,                    # location
   url_.find("h2", attrs={"class": "latest__title"}).find('a')["href"])          # url
    for url_ in soup_page.find_all("div", attrs={"class": "latest__right"})]

# -- iterate to the next page
list_of_urls_dict = {page: all_urls}

while True:
  page += 1

  parent_div = browser.find_elements_by_xpath("//div[@class='paging__wrap clearfix']")
  #print(f"   parent_ul: {parent_ul}")
  element_list = parent_div[0].find_elements_by_xpath("//div[@class='paging__item']")
  element_text = [element.text for element in element_list]

  next_element = element_list[-1]
  next_group_page_link = next_element.find_elements_by_xpath(".//a")[0].get_attribute("href")
  print(f"   next_group_page_link: {next_group_page_link}")

  while True:
    try:
      browser.get(next_group_page_link)
      browser.implicitly_wait(2)   # <--- it has to be adjusted depending on your internet connection speed
      break
    except WebDriverException:
      continue

  print(f"Get all URLs (with its date and time) in page: {page:03d}")
  elem = browser.find_element_by_xpath("//*")
  source_code = elem.get_attribute("outerHTML")

  soup_page = BeautifulSoup(source_code, parser="lxml")

  all_urls = [
    (url_.find("date", attrs={"class": "latest__date"}).text,                      # date
     url_.find("h4", attrs={"class": "latest__subtitle"}).text,                    # location
     url_.find("h2", attrs={"class": "latest__title"}).find('a')["href"])          # url
      for url_ in soup_page.find_all("div", attrs={"class": "latest__right"})]
  list_of_urls_dict[page] = all_urls

  t_last_url = all_urls[-1][0]
  t_last_url = t_last_url.split(",")[0].split()
  t_last_url[1] = month_dict[t_last_url[1]]
  t_last_url = "/".join(t_last_url)
  t_last_url = datetime.datetime.strptime(t_last_url, "%d/%m/%Y")
  print(f"   {t_last_url.strftime('%Y/%m/%d')}")

  if t_last_url < t_start:
    break

Go to the URL
url: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa
Get all URLs (with its date and time) in page: 001
   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=2
Get all URLs (with its date and time) in page: 002
   2021/03/01
   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=3
Get all URLs (with its date and time) in page: 003
   2020/11/02
   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=4
Get all URLs (with its date and time) in page: 004
   2020/10/09
   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=5
Get all URLs (with its date and time) in page: 005
   2020/08/13
   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=6
Get all URLs (with its date and time) in page: 006
   2020/05/31
   next_group_page_link: https://www.pikiran-rakyat.com/search/?q=unjuk+rasa&page=7
Get all URLs (with its date and time) in page: 00

In [77]:
list_of_urls_dict

{1: [('31 Januari 2022, 18:02 WIB',
   'Jawa Barat',
   'https://www.pikiran-rakyat.com/jawa-barat/pr-013614173/ketua-umum-hingga-penunggang-patung-macan-lodaya-belasan-anggota-gmbi-ditetapkan-sebagai-tersangka'),
  ('29 Januari 2022, 07:41 WIB',
   'Internasional',
   'https://www.pikiran-rakyat.com/internasional/pr-013594340/unjuk-rasa-tuntut-hak-atas-air-berujung-ricuh-warga-iran-serang-truk-afghanistan'),
  ('29 Januari 2022, 06:49 WIB',
   'Jawa Barat',
   'https://www.pikiran-rakyat.com/jawa-barat/pr-013594019/dikumpulkan-di-polda-jateng-buntut-kerusuhan-di-polda-jabar-anggota-gmbi-alhamdulillah-dapat-makan'),
  ('28 Januari 2022, 15:52 WIB',
   'Jawa Barat',
   'https://www.pikiran-rakyat.com/jawa-barat/pr-013590001/polres-sumedang-sedihnya-jadi-anggota-gmbi-sumedang-anak-buah-susah-pimpinannya-ngumpet'),
  ('28 Januari 2022, 15:10 WIB',
   'Bandung Raya',
   'https://www.pikiran-rakyat.com/bandung-raya/pr-013589695/buntut-aksi-ricuh-di-bandung-polisi-tangkap-ketua-gmbi'),
  ('2

In [78]:
# Save data
# with open(f"[{query}]-pikiran-rakyat_2019_2022.pkl", "wb") as data:
#   pickle.dump(list_of_urls_dict, data)

In [79]:
# Load data
with open(f"[{query}]-pikiran-rakyat_2019_2022.pkl", "rb") as data:
  loaded_list_of_urls_dict =  pickle.load(data)

## Parsing only specific year and scraping the news

In [80]:
selected_year = 2021

month_dict = {
  "Januari": "01", "Februari": "02", "Maret": "03",
  "April": "04", "Mei": "05", "Juni": "06",
  "Juli": "07", "Agustus": "08", "September": "09",
  "Oktober": "10", "November": "11", "Desember": "12"}

list_of_urls_selected_year = []
for k, v in loaded_list_of_urls_dict.items():
  for dt, loc, news_url in v:
    t_news = dt.split(",")[0].split()
    t_news[1] = month_dict[t_news[1]]
    t_news = "/".join(t_news)
    t_news = datetime.datetime.strptime(t_news, "%d/%m/%Y")

    dt = t_news.strftime("%Y/%m/%d")
    if t_news.year == 2021:
      list_of_urls_selected_year.append((dt, loc, news_url))


list_of_urls_selected_year

[('2021/12/13',
  'Nasional',
  'https://www.pikiran-rakyat.com/nasional/pr-013224091/penjaga-toilet-unjuk-rasa-sebab-wc-spbu-gratis-rocky-gerung-karena-keinginan-erick-thohir'),
 ('2021/11/26',
  'Nasional',
  'https://www.pikiran-rakyat.com/nasional/pr-013098599/polisi-amankan-peluru-tajam-saat-tangkap-15-anggota-pemuda-pancasila'),
 ('2021/11/25',
  'Jawa Barat',
  'https://www.pikiran-rakyat.com/jawa-barat/pr-013091911/sambut-keputusan-mk-soal-uu-cipta-kerja-ribuan-buruh-jabar-unjuk-rasa-ke-jakarta-dan-gedung-sate'),
 ('2021/10/28',
  'Nasional',
  'https://www.pikiran-rakyat.com/nasional/pr-012892066/evaluasi-kepemimpinan-presiden-jokowi-massa-buruh-akan-kepung-istana-negara-siang-ini'),
 ('2021/10/23',
  'Nasional',
  'https://www.pikiran-rakyat.com/nasional/pr-012857542/potret-polisi-asyik-kongkow-bareng-mahasiswa-di-sela-demo-beredar-netizen-kalau-gini-kan-enak'),
 ('2021/10/16',
  'Nasional',
  'https://www.pikiran-rakyat.com/nasional/pr-012809645/sebut-oknum-polisi-yang-banti

### Test for one url entry

In [86]:
nth_url_entry = 11

first_entry_response = requests.get(list_of_urls_selected_year[nth_url_entry][2])
print(first_entry_response.status_code, first_entry_response.url)

first_entry_page_texts = []

while True:
  first_entry_html_page = first_entry_response.content
  first_entry_soup_page = BeautifulSoup(first_entry_html_page, parser="lxml")

  first_entry_content = first_entry_soup_page.find("article", attrs={"class": "read__content clearfix"})

  for tag in first_entry_content.find_all("p"):
    par_text = tag.get_text()

    if "Baca Juga" in par_text:
      tag.replace_with("")

  # remove <a>
  for tag in first_entry_content.find_all("a"):
    tag.replace_with(tag.get_text())

  # remove <center>
  for tag in first_entry_content.find_all("center"):
    tag.replace_with("")

  # convert into string
  first_entry_pars = first_entry_content.prettify(formatter="html")

  # remove HTML tag  (it removes after it output into string)
  CLEANR = re.compile("<.*?>")
  first_entry_pars = re.sub(CLEANR, "", first_entry_pars).strip()

  # remove \n, \r, and &nbsp;
  first_entry_pars = first_entry_pars.replace("\n", "")
  first_entry_pars = first_entry_pars.replace("\r", "")
  first_entry_pars = first_entry_pars.replace("&nbsp;", "")
  first_entry_pars = first_entry_pars.replace("&ndash;", "-")
  first_entry_pars = first_entry_pars.replace("&lsquo;", "'")
  first_entry_pars = first_entry_pars.replace("&rsquo;", "'")
  first_entry_pars = first_entry_pars.replace("&ldquo;", "\"")
  first_entry_pars = first_entry_pars.replace("&rdquo;", "\"")
  first_entry_pars = first_entry_pars.replace("    ", " ")
  first_entry_pars = first_entry_pars.replace("   ", " ")
  first_entry_pars = first_entry_pars.replace("  ", " ")
  first_entry_pars = first_entry_pars.replace(".  ", ". ")
  first_entry_pars = first_entry_pars.replace("  .", ".")
  first_entry_pars = first_entry_pars.replace(" .", ".")
  first_entry_pars = first_entry_pars.replace(" ,", ",")
  first_entry_pars = first_entry_pars.replace(". \"", ".\"")
  first_entry_pars = first_entry_pars.replace("( ", "(")
  first_entry_pars = first_entry_pars.replace(" )", ")")

  first_entry_page_texts.append(first_entry_pars)

  # Check if "Selanjutnya" button exists
  get_selanjutnya = first_entry_soup_page.find("div", attrs={"class": "paging__all"})
  if get_selanjutnya is None:
    print("end of page")
    break

  get_selanjutnya = get_selanjutnya.find('a')["href"]

  first_entry_response = requests.get(get_selanjutnya)


print(f"num. of page divisions: {len(first_entry_page_texts)}")
first_entry_page_texts = " ".join(first_entry_page_texts)

first_entry_page_texts
  

200 https://www.pikiran-rakyat.com/nasional/pr-012748932/layangkan-surat-ke-mpr-komando-akan-lakukan-unjuk-rasa-desak-kepastian-pancasila
end of page
num. of page divisions: 2


'PIKIRAN RAKYAT - Aliansi mahasiswa yang tergabung dalam Konsolidasi Mahasiswa Nasional Indonesia (KOMANDO) berencana menggelar aksi unjuk rasa di depan Gedung DPR dan atau MPR pada hari ini Kamis, 7 Oktober 2021 untuk menuntut Pancasila menjadi sumber hukum tertinggi. Sebelum menggelar aksi unjuk rasa, Konsolidasi Mahasiswa Nasional Indonesia (KOMANDO) telah melayangkan surat kepada Pimpinan MPR Periode 2019 - 2024 pada momentum peringatan Hari Kesaktian Pancasila, 1 Oktober 2021. Dalam surat tersebut, KOMANDO mendesak pimpinan MPR Periode 2019 - 2024 untuk menemui mahasiswa pada gerakan aksi 7 Oktober 2021 yang akan menyuarakan beberapa tuntutan, yaitu: 1. Mendesak pembahasan penataan sistem hukum dan peraturan perundang undangan berdasarkan Pancasila sebagai sumber dari segala sumber hukum. 2. Memastikan Pancasila memiliki fungsi hukum yang mengikat dan memaksa. 3. Meletakkan Pancasila sebagai hirarki tertinggi wujud dari kepastian Pancasila sumber dari segala sumber hukum. Presidiu

### Get all texts from all urls 

In [90]:
def get_text_from_url(valid_url):
  response = requests.get(valid_url)
  print(f"   {response.status_code} {response.url}")

  page_texts = []

  while True:
    html_page = response.content
    soup_page = BeautifulSoup(html_page, parser="lxml")

    content = soup_page.find("article", attrs={"class": "read__content clearfix"})

    for tag in content.find_all("p"):
      par_text = tag.get_text()

      if "Baca Juga" in par_text:
        tag.replace_with("")

    # remove <a>
    for tag in content.find_all("a"):
      tag.replace_with(tag.get_text())

    # remove <center>
    for tag in content.find_all("center"):
      tag.replace_with("")

    # convert into string
    pars = content.prettify(formatter="html")

    # remove HTML tag  (it removes after it output into string)
    CLEANR = re.compile("<.*?>")
    pars = re.sub(CLEANR, "", pars).strip()

    # remove \n, \r, and &nbsp;
    pars = pars.replace("\n", "")
    pars = pars.replace("\r", "")
    pars = pars.replace("&nbsp;", "")
    pars = pars.replace("&ndash;", "-")
    pars = pars.replace("&lsquo;", "'")
    pars = pars.replace("&rsquo;", "'")
    pars = pars.replace("&ldquo;", "\"")
    pars = pars.replace("&rdquo;", "\"")
    pars = pars.replace("    ", " ")
    pars = pars.replace("   ", " ")
    pars = pars.replace("  ", " ")
    pars = pars.replace(".  ", ". ")
    pars = pars.replace("  .", ".")
    pars = pars.replace(" .", ".")
    pars = pars.replace(" ,", ",")
    pars = pars.replace(". \"", ".\"")
    pars = pars.replace("( ", "(")
    pars = pars.replace(" )", ")")

    page_texts.append(pars)

    # Check if "Selanjutnya" button exists
    get_selanjutnya = soup_page.find("div", attrs={"class": "paging__all"})
    if get_selanjutnya is None:
      print("end of page")
      break

    get_selanjutnya = get_selanjutnya.find('a')["href"]

    response = requests.get(get_selanjutnya)


  print(f"num. of page divisions: {len(page_texts)}")
  page_texts = " ".join(page_texts)

  return page_texts

In [91]:
news_data = []
for dt, loc, valid_url in list_of_urls_selected_year:  
  print(dt, loc)
  news_data.append((dt, loc, get_text_from_url(valid_url)))

2021/12/13 Nasional
   200 https://www.pikiran-rakyat.com/nasional/pr-013224091/penjaga-toilet-unjuk-rasa-sebab-wc-spbu-gratis-rocky-gerung-karena-keinginan-erick-thohir
end of page
num. of page divisions: 2
2021/11/26 Nasional
   200 https://www.pikiran-rakyat.com/nasional/pr-013098599/polisi-amankan-peluru-tajam-saat-tangkap-15-anggota-pemuda-pancasila
end of page
num. of page divisions: 3
2021/11/25 Jawa Barat
   200 https://www.pikiran-rakyat.com/jawa-barat/pr-013091911/sambut-keputusan-mk-soal-uu-cipta-kerja-ribuan-buruh-jabar-unjuk-rasa-ke-jakarta-dan-gedung-sate
end of page
num. of page divisions: 2
2021/10/28 Nasional
   200 https://www.pikiran-rakyat.com/nasional/pr-012892066/evaluasi-kepemimpinan-presiden-jokowi-massa-buruh-akan-kepung-istana-negara-siang-ini
end of page
num. of page divisions: 2
2021/10/23 Nasional
   200 https://www.pikiran-rakyat.com/nasional/pr-012857542/potret-polisi-asyik-kongkow-bareng-mahasiswa-di-sela-demo-beredar-netizen-kalau-gini-kan-enak
end of p

## Save the results

In [92]:
with xlsxwriter.Workbook(f"{selected_year}-[{query}]-pikiran-rakyat.xlsx") as workbook:
    worksheet = workbook.add_worksheet()

    for row_num, (dt, loc, news_text) in enumerate(news_data):
        worksheet.write(row_num, 0, dt)
        worksheet.write(row_num, 1, loc)
        worksheet.write(row_num, 2, news_text)